In [14]:
import re
import time
from config import username, password
from splinter import Browser
from Models.DB import DB

In [2]:
db = DB()
db.connect()
db.createTableReviews()
db.createTableResponses()

You are connected to -  ['PostgreSQL 11.5 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.3 20140911 (Red Hat 4.8.3-9), 64-bit'] 

(['companyreviews', 'public', 'reviews', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None], ['companyreviews', 'public', 'responses', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None])
(['companyreviews', 'public', 'reviews', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None], ['companyreviews', 'public', 'responses', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None])


In [3]:
# db.dropTables("reviews", "responses")

()


In [9]:
urls = ["https://www.glassdoor.com/index.htm","https://www.glassdoor.com/Reviews/Daugherty-Business-Solutions-Reviews-E233317.htm"]

In [17]:

def login(url=urls[0]):
    """
        This function doesn't work because Glassdoor uses bot protection (i.e. must manually login before running "scrapeReviews")
        function.
    """
    global browser
    
    executable_path = {'executable_path': 'chromedriver.exe'}
    browser = Browser('chrome', **executable_path, headless=False)
    browser.driver.set_window_size(1280, 720)
    
    browser.visit(url)
    
    button = browser.links.find_by_partial_href("/profile/login_input.htm?userOriginHook=HEADER_SIGNIN_LINK")
    
    browser.visit( button._element.get_attribute("href"))
    
    browser.fill('username', username)
    browser.fill('password', password)
    button = browser.find_by_css("button[class='gd-ui-button minWidthBtn css-1sdotxz']")
    button.click()

In [18]:
#login()

In [22]:
def visitURL(url=urls[1]):
    global browser
    
    executable_path = {'executable_path': 'chromedriver.exe'}
    browser = Browser('chrome', **executable_path, headless=False)
    browser.driver.set_window_size(1280, 720)
    
    browser.visit(url)

In [23]:
visitURL(url=urls[1])

In [24]:
def scrapeReviews():
    """
        header, body, rating, role, date, source
    """
    
    _id = 0
    
    while browser.is_element_present_by_css("a[class='pagination__ArrowStyle__nextArrow  ']"):
        
        reviews = browser.find_by_css("div[class='hreview']")
        button = browser.find_by_css("a[class='pagination__ArrowStyle__nextArrow  ']")
        
        for review in reviews:

            _id += 1
            
            header = review.find_by_css("a[class='reviewLink']").text
            body = review.find_by_css("div[class='col-sm-11 pl-sm-lg mx-0']").text
            rating = float(review.find_by_css('span[class="value-title"]')._element.get_attribute('title'))
            role = review.find_by_css('span[class="authorJobTitle middle reviewer"]').text
            try:
                date = review.find_by_css("time[class='date subtle small']").text
            except:
                date = "0"
            source = "glassdoor"
            
            
            print(f"Id: {_id}\nDate: {date}\n{header}\n{rating}\n{body}\n{source}\n-----------------------------------------------------------------------------")
            
            db.intoReviews(header, body, rating, role, date, source)
            
        #simulates clicking "next" button
        browser.visit( button._element.get_attribute("href"))

In [ ]:
def scrapeResponses(review, _id):
    """
        body, role, date, source, post_id
    """
    pass

In [25]:
scrapeReviews()

Id: 1
Date: 0
"Invests in its employees and you can tell"
5.0
"Invests in its employees and you can tell"
Current Employee - Consultant in Saint Louis, MO
Recommends
Positive Outlook
Approves of CEO
I have been working at Daugherty Business Solutions full-time for more than 8 years
Pros
- Very supportive of its employees. Daugherty continuously invests in its employees. The ICAgile partnership catapulted my career. Unmatched training program!
- Great benefits that make it as easy as possible to work while raising a family. Daugherty leadership makes this a priority each year and you can tell. 
- Really appreciate the Health &amp; Wellness program, and the free screenings, fitness center, and...
Show More
Cons
- Some growing pains as we climbed over 1,200 employees and highest revenues ever but I think employees understand there’s a balance when you have an entrepreneurial culture.
Advice to Management
- Continue to invest in current employees, it’s been fun watching friends take on mor

ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

In [ ]:
browser.visit( button._element.get_attribute("href"))